In [ ]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

In [ ]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets

In [ ]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


In [ ]:
from EarlyExitModel import EarlyExitModel

def create_model_with_exits(model_type, num_classes):
    if model_type == "resnet":
        resnet = models.resnet50(pretrained=True)
        resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
        
        model = EarlyExitModel(resnet, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer) for layer in ('layer1', 'layer2', 'layer3')]
        
    elif model_type == "vgg":
        vgg = models.vgg11(pretrained=True)
        vgg.classifier[-1] = nn.Linear(vgg.classifier[-1].in_features, num_classes)
        
        model = EarlyExitModel(vgg, num_classes, device)
        model.clear_exits()
        exit_layers = [model.add_exit(layer, model_type) for layer in ('6', '11', '16', '20')]
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model.to(device)
    return model
        
        

In [ ]:
# choose the model
model_type = "vgg" # either "resnet" or "vgg"

model_path = os.path.join("models", model_type)

model = create_model_with_exits(model_type, num_classes=10)

model

In [ ]:
from DataLoader import CustomDataset

if model_type == "resnet":
    # use the imagenette dataset
    hf_dataset = load_dataset("frgfm/imagenette", '320px')
    hf_dataset = concatenate_datasets(hf_dataset.values())
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

elif model_type == "vgg":
    # use the cifar10 dataset
    hf_dataset = load_dataset("cifar10")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])

torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32 if model_type == "resnet" else 64

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

## Early Exit Model Training

If you want tensorboard support, you need to run the following commands on **macOS**:

This clears your logs:
```
rm -rf runs/*
```

This shows your public IP address:
```
dig -4 TXT +short o-o.myaddr.l.google.com @ns1.google.com
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}/runs
```
```



If you want tensorboard support, you need to run the following commands on **Windows**:

This clears your logs:
```
rmdir /s /q runs
```

This shows your IP address:
```
powershell -command "$ipAddress = (Invoke-WebRequest -Uri 'http://ipinfo.io/ip').Content.Trim(); Write-Host 'Your IP address is: ' $ipAddress"  
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}\runs
```

In [ ]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device, model_dir=model_path)

In [ ]:
# train the classifiers
trainer.train_classifiers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)


In [ ]:
# train the exits

#trainer.train_exit_layers(train_dataloader, epoch_count=30, validation_loader=test_dataloader)